# Importing libraries

In [1]:
import pickle
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [6]:
import tensorflow
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Reshape, MaxPooling2D,InputLayer, Input,Conv2D, Dense, Flatten,Dropout
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

 # reading train-test files

In [3]:
#Source:https://stackoverflow.com/questions/35067957/how-to-read-pickle-file

with open('train_image.pkl', 'rb') as f:
    train_set  = pickle.load(f)

In [4]:
with open('test_image.pkl', 'rb') as f:
    test_set  = pickle.load(f)

In [12]:
with open('train_label.pkl', 'rb') as f:
    train_labels = pickle.load(f)

 # train-test preprocessing

In [14]:
le.fit(train_labels)                 
#transforming train labels
train_label=le.transform(train_labels)   
print(np.unique(train_label))
train_l=pd.DataFrame(train_label)

[0 1 2 3]


In [16]:
#Reading all data to dataframe

train_d=pd.DataFrame(train_set)
train_l=pd.DataFrame(train_label)
test=pd.DataFrame(test_set)
train_all=pd.concat([train_d,train_l],axis=1)

In [38]:
# Train-test split into 70:30 ratio

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(train_d.values,train_l.values, test_size=0.3, random_state=42)

In [46]:
def convert_train_test(train,test,train_l,test_l):
    train = train.reshape(-1, 28,28, 1)
    test = test.reshape(-1, 28,28, 1)
    train = train.astype('float32')
    test = test.astype('float32')
    train = train / 255.
    test = test / 255.
    train_l = to_categorical(train_l)
    test_l = to_categorical(test_l)
    return train,test,train_l,test_l

In [47]:
X_train, X_test, y_train, y_test=convert_train_test(train_X, test_X, train_y, test_y)

# Model training

In [48]:
# 3 layer convolutional-network
#Source: https://www.datacamp.com/community/tutorials/convolutional-neural-networks-python

def set_params_3layers(fmnist_model,x_params,dropout_params,alpha_params):
    for i in range (len(x_param)):
        fmnist_model.add(Conv2D(x_params[i],kernel_size=(3,3),activation='linear',padding='same',input_shape=(28,28,1)))
        fmnist_model.add(LeakyReLU(alpha=alpha_params[i]))
        fmnist_model.add(MaxPooling2D((2, 2),padding='same'))
        fmnist_model.add(Dropout(dropout_params[i]))
    return fmnist_model

In [49]:
def init_variables():
    batch_size = 56
    number_of_epochs = 15
    number_of_classes = 4
    return batch_size,number_of_epochs,number_of_classes

In [50]:
x_param=[32,64,128]
dropout_param=[0.25,0.25,0.25]
alpha_param=[0.1,0.1,0.1]
batch_size,number_of_epochs,number_of_classes=init_variables()
fmnist_model = Sequential()
f_model= set_params_3layers(fmnist_model,x_param,dropout_param,alpha_param)
f_model.add(Flatten())
f_model.add(Dense(128, activation='linear'))
f_model.add(LeakyReLU(alpha=0.1))           
f_model.add(Dropout(0.25))
f_model.add(Dense(number_of_classes, activation='softmax'))

In [51]:
f_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [52]:
trained_model = f_model.fit(X_train, y_train, batch_size=batch_size,epochs=number_of_epochs,verbose=1,validation_data=(X_test, y_test))

Train on 5600 samples, validate on 2400 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
5600/5600 [==============================] - 15s 3ms/sample - loss: 0.8034 - acc: 0.6464 - val_loss: 0.6287 - val_acc: 0.7175

# Testing

In [59]:
def convert_test(test_data):
    test_data = test_data.reshape(-1, 28,28, 1)
    test_data = test_data.astype('float32')

    test_data = test_data / 255.
    return test_data

In [60]:
def testing(test,model):
    predictions= model.predict(test)
    #predictions.to_csv('results.csv')
    return predictions

In [68]:
# Re-convert test
def re_convert_test(predictions):
    predicted = np.argmax(np.round(predictions),axis=1)
    le.fit(train_labels)
    pred=le.inverse_transform(predicted)
    return pred

In [69]:
test_2=test.values
test_d=convert_test(test_2)
pred=testing(test_d,f_model)
print(pred)
pred=re_convert_test(pred)

[[  9.96958494e-01   9.54224670e-04   8.95863650e-06   2.07833899e-03]
 [  5.78758180e-01   1.53957848e-02   2.04055514e-02   3.85440439e-01]
 [  9.48293149e-01   6.46700646e-05   1.01995211e-04   5.15402518e-02]
 ..., 
 [  4.30330560e-02   2.89429468e-03   2.69412585e-02   9.27131414e-01]
 [  4.43882209e-06   1.03880884e-04   1.04023445e-06   9.99890685e-01]
 [  4.76623303e-04   1.42838026e-03   1.77037640e-04   9.97917950e-01]]


In [71]:
#To-csv
pred_labels=pd.DataFrame(pred)
pred_labels.to_csv('predicted_labels.csv')